# Introduction

The Bitrepository quickstart is a package for quickly getting a basic fully functioning bitrepository reference system up on a localhost.

The quickstart will setup a bitrepository comprised of:
* Webclient
* AuditTrail service
* Alarm service
* Integrity service
* Monitoring (status) service
* 1 checksum pillar
* 2 reference pillars

As the setup is basic – encryption, authentication and authorization is not enabled in the quickstart.

# Prerequisites and requirements

For the quickstart to work some prerequisites and requirements exists.

All the quickstart components are meant to run on the same machine in an Linux environment. Possibility accessible from other machines, provided that firewall rules allow.

## Infrastructure

The quickstart needs some infrastructure, for message exchange and file exchange

### Message bus

The bitrepository needs an Apache Active MQ for sending messages between components. In the quickstart it is assumed that it is accessible on the localhost on the default port using a tcp connection. I.e. tcp://localhost:61616

It must not require any authentication to connect.

The default settings from the Apache Active MQ destribution will suffice, but the user/deployer is free to make changes to the Active MQ installation as long as the MQ can be reached as described above.

If the MQ is not available on the above url, the pillars and services simply will not work.

Apache Active MQ can be obtained from the ActiveMQ download site.

You can run this process in docker with this command

In [ ]:
!docker run \
    --detach \
    --rm \
    --env 'ACTIVEMQ_MIN_MEMORY=512' \
    --env 'ACTIVEMQ_MAX_MEMORY=2048' \
    --publish 61616:61616  \
    --name activemq \
    webcenter/activemq:5.12.0 \
    /opt/activemq/bin/activemq console

You can stop the activemq server again with this command

In [ ]:
!docker stop activemq

### Webserver

Per design the bitrepository uses a webdav server for file transfer. The quickstart configuration assumes that a webdav server is available on http://localhost/dav/.

It must not require any authentication to connect.

If no Webdav server is available see File Exchange Server Setup for references on setup. 

Should there be a HTTP server running on localhost that does not support webdav, or that use of another webdav server is wanted, then the ReferenceSettings.xml files for the AuditTrailService and CommandLine client needs to be have their FileExchange section changed to reflect this.

As the file location is specified per request this is not a hard requirement for deployment of the quickstart package, but something that needs to be taken into consideration.

You can run a apache2 based webdav server in docker with this easy command

In [ ]:
!docker run \
    --detach \
    --publish 80:80 \
    --name webdav \
    blekinge/apache_webdav

You can stop this webdav server again with this command

In [ ]:
!docker stop webdav

## Runtime requirements

To run the quickstart a few system requirements must be in place.

A java runtime enviroment 1.8 or newer is needed.

For the quickstart.sh script, the "curl" is needed to retrieve the Tomcat servlet container.

# Running the quickstart

When having ensured the above mentioned requirements are in place the quickstart package should be obtained from: [Quickstart](https://sbforge.org/nexus/content/repositories/releases/org/bitrepository/reference/bitrepository-integration/) ([newest release](https://sbforge.org/nexus/content/repositories/releases/org/bitrepository/reference/bitrepository-integration/1.8/bitrepository-integration-1.8-quickstart.tar.gz), [devel version](https://sbforge.org/nexus/service/local/artifact/maven/redirect?g=org.bitrepository.reference&a=bitrepository-integration&v=LATEST&r=snapshots&p=tar.gz&c=quickstart&)).

The quickstart tar.gz should be unpackaged.

Via the commandline cd to the unpacked directory and run the command "./setup.sh"

The first time running the setup script will adapt the configuration files to work with the deployed destination. Thus the quickstart will stop working if the quickstart directory is moved to another destination after the first run.

Running the setup.sh script does the following:

* Adapt the configuration files to the environment (first run only)
* Create sub directories and deploy the needed components to them.
* Start the pillars
* Download a Tomcat server for services and webclient
* Deploy services and webclient to Tomcat server and start it.
* After the script has finished the system should be accessible through: http://localhost:8080/bitrepository-webclient

In [ ]:
%%bash
wget -Nq "https://sbforge.org/nexus/service/local/artifact/maven/redirect?g=org.bitrepository.reference&a=bitrepository-integration&v=LATEST&r=snapshots&p=tar.gz&c=quickstart&" -O quickstart.tgz
tar -xzf quickstart.tgz

In [ ]:
%%bash
cd bitrepository-quickstart
./setup.sh

# Sanity Tests

Test for checking the sanity of the system as a whole. This means a roundtrip with only sunshine usage of the system getting through all the core functionality

## Sunshine roundtrip

The purpose of the test is to make certain that the core functionality works as expected when behaving nicely. The test is designed such that the system state is the same prior to the test starting and after ending it - provided that the test passes. This also means that the test can be repeated an arbitrary number of times as long as it passes. 

In the following the same fileID is used in all operations. The only restriction is that the fileID is valid for all components in the test, and is not already present in the collection.

### List all fileID's to and note which files are present in the collection. 

In [ ]:
!bitrepository-quickstart/commandline/bin/bitmag.sh get-file-ids -c books

### Put a file with an allowed fileID which is not already in the collection. 
Wait for the put file operation to finish with success for all pillars. 

In [ ]:
!bitrepository-quickstart/commandline/bin/bitmag.sh put-file -c books -f bitrepository-quickstart/commandline/logback.xml

### List all files

#### Verify that the file put in 1 is present in the listing and that it is present on all (3) pillars in the collection. 

In [ ]:
!bitrepository-quickstart/commandline/bin/bitmag.sh get-file-ids -c books

#### Trigger a collection of audittrails, and verify that the new file is present in the list of audit trail events. 

http://localhost:8080/bitrepository-webclient/audit-trail-service.html

#### Trigger a full integrity check, and verify that the new file is present and consistent. 

http://localhost:8080/bitrepository-webclient/integrity-service.html 

### Request a MD5 checksum for the file.

#### Verify that all pillars deliver the requested checksum, and that all pillars agree on the checksum value. 

In [ ]:
!bitrepository-quickstart/commandline/bin/bitmag.sh get-checksums -c books -i logback.xml

### Request a salted checksum request.

#### Verify that the checksum pillar is unable to perform the operation. 

In [ ]:
!bitrepository-quickstart/commandline/bin/bitmag.sh get-checksums -c books -i logback.xml -S abcd -R MD5 -p checksum-pillar

#### Verify that all full pillars deliver and agree on the new calculation. 

In [ ]:
!bitrepository-quickstart/commandline/bin/bitmag.sh get-checksums -c books -i logback.xml -S abcd -R MD5  

#### Verify that the salted checksum differs from the non-salted. 

In [ ]:
!bitrepository-quickstart/commandline/bin/bitmag.sh get-checksums -c books -i logback.xml -R MD5
!bitrepository-quickstart/commandline/bin/bitmag.sh get-checksums -c books -i logback.xml -S 'abcd' -R MD5

### Get file from pillar

#### Verify its content is the same as the file that was previously put. 

In [ ]:
%%bash
md5sum bitrepository-quickstart/commandline/logback.xml
bitrepository-quickstart/commandline/bin/bitmag.sh get-file -c books -i logback.xml -l logback.tmp.xml
md5sum logback.tmp.xml

### Replace the file with a new file with different content than the original on each pillar. 

In [ ]:
%%bash
xmllint bitrepository-quickstart/commandline/logback.xml > logback.changed.xml
oldhash=$(md5sum bitrepository-quickstart/commandline/logback.xml | cut -d' ' -f1)
bitrepository-quickstart/commandline/bin/bitmag.sh replace-file -c books -i logback.xml -f logback.changed.xml -C $oldhash -p file1-pillar
bitrepository-quickstart/commandline/bin/bitmag.sh replace-file -c books -i logback.xml -f logback.changed.xml -C $oldhash -p file2-pillar
bitrepository-quickstart/commandline/bin/bitmag.sh replace-file -c books -i logback.xml -f logback.changed.xml -C $oldhash -p checksum-pillar

Wait for each pillar to complete the operation.

### List all files

#### Verify that the fileID is still present on all pillars. 

In [ ]:
!bitrepository-quickstart/commandline/bin/bitmag.sh get-file-ids -c books

### Calculate checksum for the file.

#### Verify that all pillars deliver and agree on the checksum. 

In [ ]:
!bitrepository-quickstart/commandline/bin/bitmag.sh get-checksums -c books -i logback.xml

#### Verify that the checksum differs from the one for the old file. 

In [ ]:
!md5sum bitrepository-quickstart/commandline/logback.xml
!bitrepository-quickstart/commandline/bin/bitmag.sh get-checksums -c books -i logback.xml

### Get file from pillar

#### Verify that the files content is the same as the file uploaded during the replace action. 

In [ ]:
!bitrepository-quickstart/commandline/bin/bitmag.sh get-file -c books -i logback.xml -l logback.downloaded.xml 
!md5sum logback.downloaded.xml 
!md5sum logback.changed.xml

### Delete the file on all pillars

Wait for the operation(s) to complete with success 

In [ ]:
%%bash
oldhash=$(md5sum logback.changed.xml | cut -d' ' -f1)
bitrepository-quickstart/commandline/bin/bitmag.sh delete -c books -i logback.xml -C $oldhash -p file1-pillar
bitrepository-quickstart/commandline/bin/bitmag.sh delete -c books -i logback.xml -C $oldhash -p file2-pillar
bitrepository-quickstart/commandline/bin/bitmag.sh delete -c books -i logback.xml -C $oldhash -p checksum-pillar

### List all files

#### Verify that the file is no longer present on any pillars. 

In [ ]:
!bitrepository-quickstart/commandline/bin/bitmag.sh get-file-ids -c books -i logback.xml

# Stopping and restarting the quickstart

Use the quickstart script to stop and start the quickstart components:

In [ ]:
!./quickstart.sh stop

Use these commands to stop the docker servers again

In [ ]:
!docker stop webdav

In [ ]:
!docker stop activemq

In [ ]:
!docker ps